# Ford-Fullkerson法(Ford-Fulkerson Algorithm)

最大フロー問題を解ける。<br>
重み付きグラフ G(V, E) に対して、流入点 s から流出点 t までの総流量を最大化する問題。
> https://www.momoyama-usagi.com/entry/math-risan15#i-6

## 計算量
O(|F|×|E|)<br>
F: 最大フロー

## 実装
1. 残余グラフ上で頂点s-t間の適当なパスを見つける
2. 見つけたパス上で流せる最大フロー F (パス上の辺の容量の最小値) を流し、残余グラフを更新する (正方向は流量 F を減算, 逆方向は流量 F を加算)
3. 手順 1 でパスが見つからなくなるまで、手順 1 と手順 2 を繰り返す

In [1]:
class FordFulkerson:
    def __init__(self, N: int):
        self.N = N
        # 残余グラフを準備
        self.G = [list() for i in range(N+1)]
    
    # 残余グラフとなる重み付き隣接頂点リストを初期化、容量 cap の辺 (u: fr, v: to) がある
    def add_edge(self, fr: int, to: int, cap):
        # forward: 頂点 fr → to に向けて増やせるフロー (初期値: cap) を格納
        forward = [to, cap, None]
        # backward: 頂点 to → fr に向けて戻せるフロー (初期値 0) を格納
        # さらに、forward[2] = backward, backward[2] = forword として、
        # 逆方向に対応する辺を取得できるようにしておく
        forward[2] = backward = [fr, 0, forward]
        # 残余グラフへ、forward, backward を格納
        self.G[fr].append(forward)
        self.G[to].append(backward)
    
    # 双方向に容量が定義される場合に使用
    def add_multi_edge(self, v1, v2, cap1, cap2):
        edge1 = [v2, cap1, None]
        edge1[2] = edge2 = [v1, cap2, edge1]
        self.G[v1].append(edge1)
        self.G[v2].append(edge2)
    
    # 深さ優先探索
    # pos: 探索のスタート地点, goal: 出口の頂点, f: 流せるフローの最大値
    def dfs(self, pos, goal, f):
        if pos == goal:
            # ゴールに到着できる経路を一つ見つけた、流量 f のフローを流せる
            return f
        # visited: 探索済みフラグ
        visited = self.visited
        visited[pos] = True
        for e in self.G[pos]:
            # nex: 次に探索のスタート地点とする pos の隣接頂点, rev: 逆方向に対応する辺
            nex, cap, rev = e
            # 容量 cap が 1 以上でかつ、まだ探索していない頂点を探索
            if cap > 0 and not visited[nex]:
                # flow: ゴールに到着して初めて、f >= 1 となる値が返される (再帰処理)
                # f = min(min(f, cap)) でゴールに到着する経路内に流せる最大フローを更新
                flow = self.dfs(nex, goal, min(f, cap))
                # 経路内の辺に対応する残余グラフを更新していく
                if flow >= 1:
                    # 増やせるフローを減算
                    e[1] -= flow
                    # 戻せるフローを加算
                    rev[1] += flow
                    # 再帰元の関数に flow を返して伝搬
                    return flow               
        # 全ての辺を探索したが、ゴールまでのフローが存在しない
        return 0
    
    # 頂点 s から頂点 t までの最大フローの総流量を返す
    def maxflow(self, s, t):
        total_flow = 0
        f = INF = 10**15
        N = self.N
        # ゴールとなる頂点 t までフローを流せる経路が見つからなくなったら終了
        while f:
            self.visited = [False] * (N+1)
            f = self.dfs(s, t, INF)
            total_flow += f
        return total_flow

## [使用例 1](https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_bp)

タンク 1 からタンク N まで毎秒最大何リットルの水を流せるか。

In [2]:
N,M = 6,7
E = [[1, 2, 5], 
     [1, 4, 4], 
     [2, 3, 4], 
     [2, 5, 7], 
     [3, 6, 3], 
     [4, 5, 3], 
     [5, 6, 5]]

ff = FordFulkerson(N)
for u,v,c in E:
    ff.add_edge(u, v, c)

ff.maxflow(1, N)

8

## [使用例 2 (燃やす埋める問題)](https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_eo)
駅 1 から N について、特急駅に指定した場合の利益を最大化する。

In [4]:
N,M=5,2
P=[3,4,-1,-5,5]
AB=[[1,3],[2,4]]
INF=10**15

S = N
T = N + 1
offset = 0
# 重み付き辺リスト
E=[]
# スタートから各頂点、各頂点からゴールへのパスを通す
for i in range(N):
    if P[i]>=0:
        # 特急駅にしない場合のコスト
        offset += P[i]
        E.append([S,i,P[i]])
    else:
        # 特急駅にする場合のコスト
        E.append([i,T,P[i]*(-1)])

for a,b in AB:
    E.append([a-1,b-1,INF])

ff=FordFulkerson(T)
for u,v,c in E:
    ff.add_edge(u, v, c)

ans=offset-ff.maxflow(S, T)
ans

7

題意の通り利益を置くと負数の取り扱いが問題になる。

全て良い方の選択肢を選び、そこからの差分をコストと見なす。<br>
コストを最小化する最小カット問題を解く。<br>
> 最小カットは、辺のコストを容量と見て s → t 間の最大流と一致するという定理があるので、最大流問題として求められる。
> https://ikatakos.com/pot/programming_algorithm/graph_theory/maximum_flow/burn_bury_problem

|                               |  1   |  2   |  3   |  4   |  5   |
| ----                          | ---- | ---- | ---- | ---- | ---- |
| s: 特急駅にしない場合のコスト  |  3   |  4   |  0   |  0   |  5   |
| t: 特急駅にする場合のコスト    |  0   |  0   |  1   |  5   |  0   |


## [使用例 3 (二部マッチング)](https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_bq)
マッチングの最大本数は、各辺の容量が 1　の最大フローを求める問題と一致。

In [6]:
N = 5
C = [['#', '.', '.', '.', '.'],
     ['#', '.', '#', '.', '.'],
     ['.', '.', '.', '.', '#'],
     ['.', '.', '.', '.', '#'],
     ['.', '.', '.', '#', '#']]

S=2*N
T=2*N+1
w=1
E=[]
for i in range(N):
    for j in range(N):
        if C[i][j]=="#":
            E.append([i,N+j,w])

for i in range(N):
    E.append([S,i,w])
    E.append([N+i,T,w])
    
ff=FordFulkerson(T)
for u,v,c in E:
    ff.add_edge(u, v, c)

ff.maxflow(S, T)

4